<a href="https://colab.research.google.com/github/bf777/MesoNet/blob/master/mesonet_demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MesoNet
Welcome to MesoNet, a toolbox for segmenting mesoscale calcium images! This notebook will take you through all the steps needed to process your own calcium imaging dataset using our models.

In [1]:
# Use tensorflow 1.x (supported by DeepLabCut)
%tensorflow_version 1.x
import tensorflow
import os
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
# Clone MesoNet repository
!git clone https://github.com/bf777/MesoNet.git

Cloning into 'MesoNet'...
remote: Enumerating objects: 992, done.
remote: Counting objects: 100% (445/445), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 992 (delta 241), reused 298 (delta 107), pack-reused 547
Receiving objects: 100% (992/992), 6.54 MiB | 24.99 MiB/s, done.
Resolving deltas: 100% (625/625), done.


## Prepare inputs

In [3]:
!mkdir /content/mesonet_inputs/

Now, we will copy sample data (11 images) from the git repository to the inputs folder. If, instead, you would like to upload your own images create a folder inside `mesonet_inputs` and put your images inside that folder; then, skip the next cell. 

In [4]:
# If you have your own input data that you've put into a subfolder under
# mesonet_inputs, don't run this cell.
%cp -r MesoNet/tests/test_input/ mesonet_inputs/

In [34]:
%cd MesoNet/

/content/MesoNet


In [37]:
# Utility to install mesonet package and associated requirements (will be replaced with pip later)
!pip install matplotlib==3.1.3

# Install DeepLabCut for pose estimation
!pip install deeplabcut

# Install MesoNet
!python setup.py install

# Reinstall OpenCV to address compatibility issue
!pip install opencv-contrib-python 
!pip install opencv-python-headless

# pip install mesonet

running install
running bdist_egg
running egg_info
writing mesonet.egg-info/PKG-INFO
writing dependency_links to mesonet.egg-info/dependency_links.txt
writing requirements to mesonet.egg-info/requires.txt
writing top-level names to mesonet.egg-info/top_level.txt
writing manifest file 'mesonet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/__init__.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/data.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/model.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/gui_start.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/gui_train.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/utils.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/img_augment.py -> bui

Because of how DeepLabCut operates, you now need to restart your runtime (under the Runtime menu).

We will now pull information from the OSF repository containing the DeepLabCut and U-Net models for our code.

In [2]:
!pip install osfclient

In [ ]:
#@title OSF Username
osf_username = 'brandon.forys@alumni.ubc.ca' #@param {type: "string"}


In [ ]:
# Get OSF info
from getpass import getpass
osf_password = getpass('OSF password')
os.environ['OSF_USER'] = osf_username
os.environ['OSF_PASSWORD'] = osf_password

In [3]:
%cd /content/
!osf -p svztu fetch 6_Landmark_estimation_model/atlas-DongshengXiao-2020-08-03.zip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

/content
100% 643M/643M [00:04<00:00, 158Mbytes/s]


In [4]:
!unzip -q mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip -d /content/mesonet_inputs

In [25]:
%cd /content/
!osf -p svztu fetch 7_U-Net_model/DongshengXiao_brain_bundary.hdf5 MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5

/content
100% 373M/373M [00:04<00:00, 89.7Mbytes/s]


Now, input the information about your input and output images, as well as the U-Net and DeepLabCut models that you would like to use. The default values will use the test data that we've included in the MesoNet git repository (in `MesoNet/mesonet/tests/test_input`). If you're using your own input data, replace `input_filename` below with the name of the folder in `mesonet_inputs` containing your input data.

In [57]:
#@title Input information for the model
input_filename = 'test_input'  #@param {type: "string"}
output_filename = 'mesonet_outputs'  #@param {type: "string"}
model_name = 'DongshengXiao_brain_bundary.hdf5' #@param {type: "string"}
dlc_model_name = 'atlas-DongshengXiao-2020-08-03'  #@param {type: "string"}

In [58]:
# Set up filepaths based on your inputs
input_file = os.path.join('/content','mesonet_inputs', input_filename)
output_file = os.path.join('/content', output_filename)
model = os.path.join('/content', 'MesoNet', 'mesonet', 'models', model_name)
dlc_config = os.path.join('/content','mesonet_inputs', dlc_model_name, 
                          'config.yaml')

In [59]:
!mkdir '/content/mesonet_outputs/'

mkdir: cannot create directory ‘/content/mesonet_outputs/’: File exists


Now that we've told Colab where to find the input and output folders, let's define the configuration file!

In [60]:
# We need to make sure that DeepLabCut doesn't run with a GUI (which isn't
# supported in Colab).
os.environ["DLClight"]="True"

# Import mesonet and define the configuration file according to your chosen
# inputs
import mesonet
config_file = mesonet.config_project(input_file, output_file, 'test',
                                     use_voxelmorph=False, 
                                     mat_save=False, config=dlc_config, 
                                     model=model)

/content/MesoNet/mesonet/


The config file (by default in `mesonet_outputs`) contains information about how MesoNet will run. We'll be using this config file as an input to the last two functions needed to run MesoNet.

In [61]:
config_file

'/content/mesonet_outputs/mesonet_test_config.yaml'

Next, we'll use the U-net model to identify the outer edges of the cortex:

In [54]:
%cd /content/
mesonet.predict_regions(config_file)

/content
/content/MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5
/content/mesonet_inputs/test_input
12/12 [==============================] - 5s 390ms/step


In [55]:
# Set this environment variable to help MesoNet find the git repo location
os.environ["MESONET_GIT"]='/content/MesoNet/mesonet/'

Lastly, we will identify and use cortical landmarks to align the atlas to the brain:

In [ ]:
mesonet.predict_dlc(config_file)

Congratulations, you're all done! You can now check the outputs in the `mesonet_outputs` folder. The segmented brain data can be found in `mesonet_outputs/output_mask`!